# Project 5 : Implementing a Recommender System on Spark
#### date: 2019-07-09
### by: Sang Yoon (Andy) Hwang, Santosh Cheruku, Anthony Munoz, N.Hwang

In [2]:
import pandas as pd
import numpy as np
from surprise import accuracy
from surprise import KNNBaseline
import surprise
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split


from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

The dataset set was on CSV file and later we download the dataset and uploaded it onto Databricks database environments system.

In [4]:
# code to find down where is the dataset file location on Databricks 
import os
for dicrectory in os.listdir('/dbfs/FileStore/tables'):
  print(dicrectory)

## Dataset & set up

For this Project, we will be using a previews project recommender system in order to compare the result later on using Spark (pySpark).
The Dataset used for this project is MovieLens, which contains a collection of movies with its user rating.
we will split the work into two sections. the first section will be using python dataframes and the second section will be using SPark (pySpark) dataframes.

##### Python Dataframe section

In [8]:
# upload the data from database file system and put it to dataframe 
movie_df = pd.read_csv("/dbfs/FileStore/tables/movies.csv", header='infer')
rating_df = pd.read_csv("/dbfs/FileStore/tables/ratings.csv", header='infer', nrows=1000000)


In [9]:
#checkign the information on the rating dataframe
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
userId 1000000 non-null int64
movieId 1000000 non-null int64
rating 1000000 non-null float64
timestamp 1000000 non-null int64
dtypes: float64(1), int64(3)
memory usage: 30.5 MB

In [10]:
#checkign the information on the movies dataframe
rating_df.info()
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId 27278 non-null int64
title 27278 non-null object
genres 27278 non-null object
dtypes: int64(1), object(2)
memory usage: 639.4+ KB

### Splitting data 80/20

In [12]:
#splitting the data 80/20
reader = Reader(rating_scale=(1, 5))
dat_new = Dataset.load_from_df(rating_df[['userId', 'movieId', 'rating']], reader=reader)
trainset, testset = train_test_split(dat_new, test_size=.20)

### Data Trainig - Alternative least square (ALS)

For this recommender system, we will be using collaborative filtering and because we can use on both python and Spark we select using the alternative least square (ALS) algorithm.

In [15]:
#ALS
bsl_options = {'method': 'als'}
algo = surprise.BaselineOnly(bsl_options=bsl_options)

# Train the algorithm on the trainset, and predict ratings for the testset
predictions_als = accuracy.rmse(algo.fit(trainset).test(testset))


Estimating biases using als...
RMSE: 0.8625

##### Spark section

In [17]:
# reading the data to spark dataframe
movie_sdf = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/movies.csv')
rating_sdf = spark.read.format('csv').options(header='true', inferSchema='true' ).load('/FileStore/tables/ratings.csv' )


In [18]:
#taking a look how are the Spark dataframe schema
rating_sdf.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: integer (nullable = true)

#### Splitting data 80/20

In [20]:
Sdf = rating_sdf.select(['userId', 'movieId', 'rating'])
train, test = Sdf.randomSplit([0.8,0.2])

#### Rating count summary

In [22]:
display(rating_sdf.select(['rating']))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-535492245222866> in <module> () 
 ----> 1 dispplay ( rating_sdf . select ( [ 'rating' ] ) ) 

 NameError : name 'dispplay' is not defined

### Data Trainig - Alternative least square (ALS)

In [24]:
#creaating the ALS model predictions
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

predictions = als.fit(train).transform(test).na.drop()


In [25]:
#showing the model predictions summary
predictions.describe().show()

+-------+-----------------+------------------+-----------------+------------------+
summary| userId| movieId| rating| prediction|
+-------+-----------------+------------------+-----------------+------------------+
 count| 3996727| 3996727| 3996727| 3996727|
 mean|69026.86870481772| 9008.085115395672|3.525253413605683|3.4174060526111374|
 stddev| 40042.4260676503|19725.568379616067|1.051964442325952|0.7229671551406042|
 min| 1| 1| 0.5| -6.751219|
 max| 138493| 131013| 5.0| 9.817508|
+-------+-----------------+------------------+-----------------+------------------+

In [26]:
rmse = evaluator.evaluate(predictions)
rmse

Out[ 41 ]: 0.8150576238969827

In [27]:
print("Python ALS rmse: " + str(predictions_als) , "pySpark ALS rmse:" + str(rmse))

Python ALS rmse: 0.8625308707114622 pySpark ALS rmse:0.8150576238969827

## conclusion

The first problems to find down was the amount for rows that we can work with when using python dataframe against ALS algorithm training data,
 on this dataset we have about 2M rows and when we try to run the recommender system again he whole dataset the compile crash maybe due to the bid amount of data and all the calculations. so for a quick solution, we reduce the amount of data to a point on where the program could run and that was about 1m rows.
 
 On the Spark side, we noticed that it's really good when it comes to working with a big dataset. The training algorithm worked well with the whole dataset without crashing when doing the calculations.
 
 The python dataset which was about 1M rows this complete the training model in about 12.84 seconds which was very fast and give an RMSE accuracy of 0.8625. on the otherside with Spark, we use the whole dataset 2M and it did calculate the model in about 1.10 minutes with an RMSE accuracy of 0.8150.
 
 if we look at the RMSE we can see that Spark give better result even that it takes a little bit more time which can be debating due that have to work with the whole dataset which was twice the sizes of the python dataframe algorithm model analysis.

  we can see the advantages of Spark comes when we need to work with Big dataset because it can work very fast and with good accuracy when applying machine learning algorithm.

### Source/References

1. https://surprise.readthedocs.io/en/v1.0.0/_modules/surprise/dataset.html
 2. https://surprise.readthedocs.io/en/v1.0.0/dataset.html
 3. http://www.3leafnodes.com/apache-spark-introduction-recommender-system